# Install necessary packages

In [ ]:
%pip install -r requirements.txt

# Part 1: Introduction to Classification & Evaluation

**Objective:** Load the synthetic health data, train a Logistic Regression model, and evaluate its performance.

## 1. Setup

Import necessary libraries.

In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer

## 2. Data Loading

Implement the `load_data` function to read the dataset.

In [6]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    df = pd.read_csv(file_path)
    return df

## 3. Data Preparation

Implement `prepare_data_part1` to select features, split data, and handle missing values.

In [7]:
def prepare_data_part1(df, test_size=0.2, random_state=42):
    """
    Prepare data for modeling: select features, split into train/test sets, handle missing values.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Select relevant features (age, systolic_bp, diastolic_bp, glucose_level, bmi)
    # 2. Select target variable (disease_outcome)
    # 3. Split data into training and testing sets
    # 4. Handle missing values using SimpleImputer
    

    X = df[['age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi']]
    Y = df[['disease_outcome']]

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state = random_state)

    imputer = SimpleImputer(strategy = 'mean')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)
    
    return X_train, X_test, y_train, y_test

## 4. Model Training

Implement `train_logistic_regression`.

In [8]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    log_mod = LogisticRegression()
    return log_mod.fit(X_train, y_train)  

## 5. Model Evaluation

Implement `calculate_evaluation_metrics` to assess the model's performance.

In [9]:
def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    con_mat = confusion_matrix(y_test, y_pred)

    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'confusion_matrix': con_mat
    }
    
    return results

## 6. Save Results

Save the calculated metrics to a text file.

In [11]:
# Create results directory and save metrics
# YOUR CODE HERE
def save_results(metrics: dict):
    os.makedirs('results', exist_ok=True)
    formatted_metrics = [f"{key}: {value}" for key, value in metrics.items()]

    output_file = 'results/results_part1.txt'
    with open(output_file, 'w') as f:
        for line in formatted_metrics:
            f.write(line + '\n')
    print(f"\nMetrics saved to {output_file}")

In [12]:
## 8. Interpret Results

# Implement a function to analyze the model performance on imbalanced data.

# ```python
def interpret_results(metrics):
    """
    Analyze model performance on imbalanced data.
    
    Args:
        metrics: Dictionary containing evaluation metrics
        
    Returns:
        Dictionary with keys:
        - 'best_metric': Name of the metric that performed best
        - 'worst_metric': Name of the metric that performed worst
        - 'imbalance_impact_score': A score from 0-1 indicating how much
          the class imbalance affected results (0=no impact, 1=severe impact)
    """
    # YOUR CODE HERE
    # 1. Determine which metric performed best and worst
    # 2. Calculate an imbalance impact score based on the difference
    #    between accuracy and more imbalance-sensitive metrics like F1 or recall
    # 3. Return the results as a dictionary
    
    simple_metrics = {name: value for name, value in metrics.items() if name != 'confusion_matrix'}
    
    # Step 2: Find the metric with the highest and lowest value
    best_metric_name = max(simple_metrics, key=simple_metrics.get)
    worst_metric_name = min(simple_metrics, key=simple_metrics.get)
    
    # Step 3: Calculate the imbalance impact score
    # If accuracy is much higher than recall or F1, it could mean class imbalance
    accuracy = metrics.get('accuracy', 0)
    recall = metrics.get('recall', 0)
    f1 = metrics.get('f1', 0)
    
    # Compute how different accuracy is from recall and F1
    difference_recall = abs(accuracy - recall)
    difference_f1 = abs(accuracy - f1)
    
    # Average the two differences and cap at 1.0
    imbalance_score = (difference_recall + difference_f1) / 2
    imbalance_score = round(min(imbalance_score, 1.0), 3) 
    return {
        'best_metric': best_metric_name,
        'worst_metric': worst_metric_name,
        'imbalance_impact_score': imbalance_score
    }

## 7. Main Execution

Run the complete workflow.

In [14]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data
    X_train, X_test, y_train, y_test = prepare_data_part1(df)
    
    # 3. Train model
    model = train_logistic_regression(X_train, y_train)
    
    # 4. Evaluate model
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 5. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 6. Save results
    save_results(metrics)
    
    # 7. Interpret results
    interpretation = interpret_results(metrics)
    print("\nResults Interpretation:")
    for key, value in interpretation.items():
        print(f"{key}: {value}")

/Users/jessica/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.9168
precision: 0.6615
recall: 0.3007
f1: 0.4135
auc: 0.6420

Metrics saved to results/results_part1.txt

Results Interpretation:
best_metric: accuracy
worst_metric: recall
imbalance_impact_score: 0.56


<!-- ## 8. Interpret Results

Implement a function to analyze the model performance on imbalanced data.

```python
def interpret_results(metrics):
    """
    Analyze model performance on imbalanced data.
    
    Args:
        metrics: Dictionary containing evaluation metrics
        
    Returns:
        Dictionary with keys:
        - 'best_metric': Name of the metric that performed best
        - 'worst_metric': Name of the metric that performed worst
        - 'imbalance_impact_score': A score from 0-1 indicating how much
          the class imbalance affected results (0=no impact, 1=severe impact)
    """
    # YOUR CODE HERE
    # 1. Determine which metric performed best and worst
    # 2. Calculate an imbalance impact score based on the difference
    #    between accuracy and more imbalance-sensitive metrics like F1 or recall
    # 3. Return the results as a dictionary
    
    simple_metrics = {name: value for name, value in metrics.items() if name != 'confusion_matrix'}
    
    # Step 2: Find the metric with the highest and lowest value
    best_metric_name = max(simple_metrics, key=simple_metrics.get)
    worst_metric_name = min(simple_metrics, key=simple_metrics.get)
    
    # Step 3: Calculate the imbalance impact score
    # If accuracy is much higher than recall or F1, it could mean class imbalance
    accuracy = metrics.get('accuracy', 0)
    recall = metrics.get('recall', 0)
    f1 = metrics.get('f1', 0)
    
    # Compute how different accuracy is from recall and F1
    difference_recall = abs(accuracy - recall)
    difference_f1 = abs(accuracy - f1)
    
    # Average the two differences and cap at 1.0
    imbalance_score = (difference_recall + difference_f1) / 2
    imbalance_score = round(min(imbalance_score, 1.0), 3) 
    return {
        'best_metric': best_metric_name,
        'worst_metric': worst_metric_name,
        'imbalance_impact_score': imbalance_score
    } -->